# Import Section
---

In [1]:
import shutil
import json
import os

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from ipywidgets import AppLayout, Button, Layout, Box, FloatText, Textarea, Dropdown, Label, IntSlider
from IPython.display import display, HTML
from IPython.display import clear_output
from ipyfilechooser import FileChooser

# Widgets Control Section
---

In [4]:

class train_config_and_cmds_widgets():
    def __init__(self):
        
        self.tflite_file_loc = ""
        
        form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between',
        )
        
        ### Data acquisition ###
        self.A_aq = widgets.Text(value='cy', placeholder='Type something', disabled=False)
        self.B_aq = widgets.Text(value='wing', placeholder='Type something', disabled=False)
        self.C_aq = widgets.IntText(value=200, disabled=False)
        self.D_aq = widgets.IntText(value=1, disabled=False)
        self.E_aq = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        self.F_aq = widgets.Text(value='COM7', placeholder='Type something', disabled=False)
        
        form_acquisition_items = [
            Box([Label(value = 'Person Name'), self.A_aq], layout=form_item_layout),
            Box([Label(value = 'Action Label'), self.B_aq], layout=form_item_layout),
            Box([Label(value = 'Collecting Length (points)'), self.C_aq], layout=form_item_layout),
            Box([Label(value = 'Negative Number (If Action Label is negative)'), self.D_aq], layout=form_item_layout),
            Box([Label(value = 'COM Port'), self.F_aq], layout=form_item_layout),
            Box([Label(value = 'Collect Data'), self.E_aq], layout=form_item_layout),
        ]
        
        self.form_acquisition_cmd = Box(form_acquisition_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        ### train ###
        self.A_ta = Dropdown(options=['CNN', 'CNN-S', 'LSTM'])
        self.C_ta = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        self.D_ta = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        self.E_ta = widgets.IntSlider(value=15, min=10, max=100, step=5)
        self.F_ta = widgets.IntSlider(value=16, min=16, max=256, step=16)
         
        form_train_items = [
            Box([Label(value = 'Model Type'), self.A_ta], layout=form_item_layout),
            Box([Label(value = 'Training Epochs'), self.E_ta], layout=form_item_layout),
            Box([Label(value = 'Batch Size'), self.F_ta], layout=form_item_layout),
            Box([Label(value = 'Start Training'), self.C_ta], layout=form_item_layout),
            Box([Label(value = 'Convert to c'), self.D_ta], layout=form_item_layout),
        ]
        
        self.form_output_train_cmd = Box(form_train_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        ### data prepare ###
        self.A_dp = widgets.Textarea(value='ring slope wing', placeholder='Type something', disabled=False)
        self.B_dp = widgets.Textarea(value='cy', 
                                     placeholder='Type something', disabled=False)
        self.C_dp = widgets.Text(value='out_dataset_1', placeholder='Type something', disabled=False)
        self.D_dp = widgets.IntSlider(value=128, min=32, max=224, step=16)
        self.E_dp = widgets.FloatSlider(value=0.7, min=0.4, max=0.8, step=0.1)
        self.F_dp = widgets.FloatSlider(value=0.1, min=0.05, max=0.4, step=0.05)
        self.G_dp = widgets.IntText(value=30, disabled=False)
        self.H_dp = widgets.IntText(value=50, disabled=False)
        self.I_dp = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')


        
        form_data_prepare_items = [
            Box([Label(value = 'Label Folder'), self.A_dp], layout=form_item_layout),
            Box([Label(value = 'Names'), self.B_dp], layout=form_item_layout),
            Box([Label(value = 'Output Folder'), self.C_dp], layout=form_item_layout),
            Box([Label(value = 'Sequence Length (points)'), self.D_dp], layout=form_item_layout),
            Box([Label(value = 'Train Ratio'), self.E_dp], layout=form_item_layout),
            Box([Label(value = 'Validation Ration'), self.F_dp], layout=form_item_layout),
            Box([Label(value = 'Random Seed'), self.G_dp], layout=form_item_layout),
            Box([Label(value = 'Generated Data Number(X3)'), self.H_dp], layout=form_item_layout),
            Box([Label(value = 'Data Prepare'), self.I_dp], layout=form_item_layout),
        ]
        
        self.form_data_prepare_cmd = Box(form_data_prepare_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
         ### test ###
        self.B_tt = widgets.Textarea(value='ring slope wing', placeholder='Type something', disabled=False)
        self.C_tt = widgets.IntSlider(value=128, min=32, max=224, step=16)
        self.A_tt = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
      
         
        form_test_items = [
            Box([Label(value = 'Label Folder'), self.B_tt], layout=form_item_layout),
            Box([Label(value = 'Sequence Length (points)'), self.C_tt], layout=form_item_layout),
            Box([Label(value = 'Choose the Test Setting'), self.A_tt], layout=form_item_layout),
        ]
        
        self.form_output_test_cmd = Box(form_test_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
    def move_allfiles(self, src_folder, dst_folder):
        copy_num = 0
        
        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  #copy whole folder
                shutil.move(fullpath, dst_folder)
                print("Copy finish: {}".format(f))
    
    def show_headline(self, output):
        html0= widgets.HTML(value = f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)
    
    def show_main(self):   
        
        intro_text = 'Please Choose the setting of data prepare & train'
        htmlWidget = widgets.HTML(value = f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(htmlWidget)
        
        #Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[self.form_acquisition_cmd, self.form_data_prepare_cmd,
                                    self.form_output_train_cmd, self.form_output_test_cmd]).add_class("parentstyle")
        tab_contents = ['Data Acquisition', 'Data Prepare', 'Train', 'Test']
        tab.titles = tab_contents
       
        
        #Add a custom style tag to the notebook, you can use dev tool to inspect the class names
        #display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        #accordion.set_title(0, 'Configure the Training')
        
        output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))
        
        def act_para(model_type, label,
                     person_name, out_fld, seq_len, train_ration, val_ration,
                     rand_seed, data_num, epochs, batch,
                     p_name, action_label, max_len, ne_num, COM,
                     label_test, seq_len_test):
            
            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()
        
        #------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        #------------------#
        out_inter = widgets.interactive_output(act_para, {'model_type': self.A_ta, 'label': self.A_dp, 
                                                          'person_name': self.B_dp, 'out_fld': self.C_dp, 'seq_len': self.D_dp, 
                                                          'train_ration': self.E_dp, 'val_ration': self.F_dp, 
                                                          'rand_seed': self.G_dp, 'data_num': self.H_dp,
                                                          'epochs': self.E_ta, 'batch': self.F_ta,
                                                          'p_name': self.A_aq, 'action_label': self.B_aq, 
                                                          'max_len': self.C_aq, 'ne_num': self.D_aq, 'COM': self.F_aq,
                                                          'label_test': self.B_tt, 'seq_len_test': self.C_tt 
                                                          })
        display(tab, out_inter)
        
        #------------------#
        # for labelimg cmd, move to outside of act_para to prevent keep trigering
        #------------------#
        output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))
        display(output_widgets)
        
        def on_button_clicked_dataAcq(b):
                with output_widgets:
                    clear_output()
                    print("Data acquisition")
                    self.data_acq()       
        self.E_aq.on_click(on_button_clicked_dataAcq)
        
        def on_button_clicked_dataPrepare(b):
                with output_widgets:
                    clear_output()
                    print("Data Prepare")
                    self.data_prepare()       
        self.I_dp.on_click(on_button_clicked_dataPrepare)
        
        def on_button_clicked_train(b):
                with output_widgets:
                    clear_output()
                    print("Train. . .")
                    self.run_train()       
        self.C_ta.on_click(on_button_clicked_train)
        
        def on_button_clicked_tflu(b):
                with output_widgets:
                    clear_output()
                    print("Convert to tflu. . .")
                    self.convert_tflu()       
        self.D_ta.on_click(on_button_clicked_tflu)
        
        def on_button_clicked_test(b):
            with output_widgets:
                clear_output()
                self.test_chooser()           
        self.A_tt.on_click(on_button_clicked_test)

    def test_chooser(self):
        path_fc = os.path.join(os.getcwd(), "data")
        fc = FileChooser(path_fc)
        fc.show_only_dirs = True
        fc.title = f"<b><font color='lightblue'><font size=4>Choose the dataset.</b>"
        display(fc)
        
        path_ftflite = os.path.join(os.getcwd(), "generated_model")
        f_tflite = FileChooser(path_ftflite)
        f_tflite.filter_pattern = ['*.tflite']
        f_tflite.title = f"<b><font color='lightblue'><font size=4>Choose the Tflite for testing.</b>"
        display(f_tflite)
        
        def act_test():
            input_dataset = fc.selected_path
            input_dataset = input_dataset.split("\\")[-1]
            input_tflite  = f_tflite.selected
            print("The chosen dataset: {}".format(input_dataset))
            print("The chosen tflite: {}".format(input_tflite))
            self.test_run(input_dataset, input_tflite)
            print("Finish!")
        evt = interact_manual(act_test)
        evt.widget.children[0].description = 'Start Testing'  #because there are 3 parameter of the evt
        evt.widget.children[0].button_style = 'primary'
    
    def test_run(self, input_dataset, input_tflite):
        %run train.py --folders $self.B_tt.value --out_dir $input_dataset --seq_length $self.C_tt.value \
        --test_tflite True --tflite_path $input_tflite

    def data_acq(self):
        %run visual_sparkfun_acc.py --port $self.F_aq.value --user_name $self.A_aq.value --action_label $self.B_aq.value \
        --max_len $self.C_aq.value --negative_num $self.D_aq.value
        
    def data_prepare(self):
        print(self.A_dp.value)
        %run data_prepare.py --folders $self.A_dp.value --names $self.B_dp.value --out_dir $self.C_dp.value \
        --seq_length $self.D_dp.value --train_ratio $self.E_dp.value --val_ratio $self.F_dp.value \
        --rand_seed $self.G_dp.value --neg_data_num $self.H_dp.value
        
    def run_train(self):
        %run train.py --model $self.A_ta.value --folders $self.A_dp.value --out_dir $self.C_dp.value --seq_length $self.D_dp.value \
        --epochs $self.E_ta.value --batch_size $self.F_ta.value
        
    def convert_tflu(self):
        
        tflite_path = os.path.join("generated_model", "model_{}_quantized.tflite".format(self.A_ta.value))
        output_path = os.path.join("generated_model", "model_{}_quantized.cc".format(self.A_ta.value.lower()))
        #tflite_path = r"generated_mode\\" + r"model_quantized.tflite"
        #output_path = r"magic_wand_{}.cc".format(self.A_ta.value.lower())
        print("tflite path: {}".format(tflite_path))
        print("C file path: {}".format(output_path))
        %run tflite_to_tflu.py --tflite_path $tflite_path --output_path $output_path

# Run Section
---
- The detail description of all the parameters and each step meaning is here [meaning](#id-train_evl_monitor)
- In this notebook step, you have alreay finish the dataset prepared. If not, please go to `image_dataset\create_data.ipynb`.

In [7]:
act = train_config_and_cmds_widgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the setting of data prepare & train</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…

<a id="id-train_evl_monitor"></a>
# Parameter & Steps Description
---
